# 2DEG mobility calculation using analytical formula

Ref-1: J. Bassaler, J. Mehta, I. Abid, L. Konczewicz, S. Juillaguet, S. Contreras, S. Rennesson, S. Tamariz, M. Nemoz, F. Semond, J. Pernot, F. Medjdoub, Y. Cordier, P. Ferrandis, Al-Rich AlGaN Channel High Electron Mobility Transistors on Silicon: A Relevant Approach for High Temperature Stability of Electron Mobility. Adv. Electron. Mater. 2024, 2400069. https://doi.org/10.1002/aelm.202400069

Ref-2: Zhang, J., Hao, Y., Zhang, J. et al. The mobility of two-dimensional electron gas in AlGaN/GaN heterostructures with varied Al content. Sci. China Ser. F-Inf. Sci. 51, 780–789 (2008). https://doi.org/10.1007/s11432-008-0056-7


# 1. Settings

## 1.1 Import modules

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors, cm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import AlloyParams, Mobility2DEG, Plottings, PlotQuasi3DFuns

In [4]:
save_figure_dir = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/MobilityCalc2DEG/OUTPUTS/FIGS'
savefigure=True

In [5]:
folder_post_data_ = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/NextnanoSimulations/DATAs/AlGaN_HEMT/AlGaN_AlGaN_AlN_HEMT/nnp/2DEHG_density/OneDim/'
data_file = 'sim_post_process_data.xlsx'
excel_file = f'{folder_post_data_}/{data_file}' 

## 1.2 Set physical constants

In [6]:
'''
n_2d => in nm^-2
rms_roughness => nm^-1
corr_len => nm^-1
n_dis => nm^-2
f_dis => unit less
'''
alloy_disordered_effect=1
interface_roughness_effect=1
dislocation_effect=1
deformation_potential_effect=1 
piezoelectric_effect=1
acoustic_phonon_effect=1
polar_optical_phonon_effect=1
total_mobility=1
mobility_model='Bassaler'
density_2deg = 0.1 # nm^-2
irf_rms_roughness = 0.3 # nm
irf_corr_length = 3.0 # nm
dislocation_density = 1e-4 # nm^-2
occup_dislocation = 0.3
T=300 #K

# 2. Calculate mobilities

## 2.1 GaN LFOM for normalization

In [7]:
mu2deg = Mobility2DEG(compositions=0, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')
mobility_ref = mu2deg.calculate_mobility(n_2d=0.1, rms_roughness=irf_rms_roughness, 
                                        corr_len=irf_corr_length, n_dis=dislocation_density, 
                                        f_dis=occup_dislocation, T=T,
                                        alloy_disordered_effect=alloy_disordered_effect,
                                        interface_roughness_effect=interface_roughness_effect,
                                        dislocation_effect=dislocation_effect,
                                        deformation_potential_effect=deformation_potential_effect,
                                        piezoelectric_effect=piezoelectric_effect,
                                        acoustic_phonon_effect=acoustic_phonon_effect,
                                        polar_optical_phonon_effect=polar_optical_phonon_effect,
                                        total_mobility=total_mobility,
                                        mobility_model=mobility_model)
ref_LFOM = mu2deg.calculate_figure_of_merit(0.1, np.array(mobility_ref['TOT']))

/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/MobilityCalc2DEG/mobilitypy/tutorials/../mobilitypy/src/_mobilities_2deg.py:287: RuntimeWarning: divide by zero encountered in scalar divide
  return 1e4/(self.m0_by_e_ * inv_sc)


## 2.2 AlGaN mobilities and figure-of-merit

### 2.2.1 Read 2DEG data from nextnano simulation

In [8]:
data_sheet_name =  'sim_sweep__AlContentBarrier__AlContentChannel__ThicknessAlGaNBarrier'
n_2d_df = pd.read_excel(excel_file, sheet_name=data_sheet_name, index_col=0)
density_2deg_comp = np.array(n_2d_df['2DEG_device'], dtype=float)/1e+14
channel_comp = np.array(n_2d_df['AlContentChannel'], dtype=float)

### 2.2.1 Calculate AlGaN mobilities

In [9]:
mu2deg = Mobility2DEG(compositions=channel_comp, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')

In [10]:
mobility_df = mu2deg.calculate_mobility(n_2d=density_2deg_comp, rms_roughness=irf_rms_roughness, 
                                        corr_len=irf_corr_length, n_dis=dislocation_density, 
                                        f_dis=occup_dislocation, T=T,
                                        alloy_disordered_effect=alloy_disordered_effect,
                                        interface_roughness_effect=interface_roughness_effect,
                                        dislocation_effect=dislocation_effect,
                                        deformation_potential_effect=deformation_potential_effect,
                                        piezoelectric_effect=piezoelectric_effect,
                                        acoustic_phonon_effect=acoustic_phonon_effect,
                                        polar_optical_phonon_effect=polar_optical_phonon_effect,
                                        total_mobility=total_mobility,
                                        mobility_model=mobility_model)

/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/MobilityCalc2DEG/mobilitypy/tutorials/../mobilitypy/src/_mobilities_2deg.py:258: RuntimeWarning: divide by zero encountered in scalar divide
  fact_3 = yy / ((np.exp(self.E_pop*e_charge/k_B/self.temp_) - 1) * (1+yy-np.exp(-yy)))


### 2.2.2 Calculate AlGaN lateral figure-of-merit

In [11]:
mobility_df['LFOM'] = mu2deg.calculate_figure_of_merit(density_2deg_comp, np.array(mobility_df['TOT']))
mobility_df['LFOMnorm'] = mobility_df['LFOM'].copy()/ref_LFOM

### 2.2.3 Combine AlGaN mobilities and lateral figure-of-merit dataframe

In [12]:
Combined_df = pd.concat([n_2d_df, mobility_df], axis=1)

## 2.3 Plottings

In [13]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities')
rescale_2deg_fact=1e13

In [14]:
Combined_df['2DEG'] =  Combined_df['2DEG_device'].copy()/rescale_2deg_fact
Combined_df['2DHG'] =  Combined_df['2DHG_device'].copy()/rescale_2deg_fact

In [15]:
plot_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT', 'LFOM', 'LFOMnorm', '2DEG']
len_plots = len(plot_mobilities)
params = {'axes.labelsize': 20,'axes.titlesize': 20,
          'xtick.labelsize':20,
                  'xtick.major.width':2,
                  'xtick.major.size':5,
                  'xtick.minor.width':2,
                  'xtick.minor.size':3,
                  'ytick.labelsize': 20,
                  'ytick.major.width':2,
                  'ytick.major.size':5,
                  'ytick.minor.width':2,
                  'ytick.minor.size':3,
                  'errorbar.capsize':2}
plt.rcParams.update(params)
plt.rc('font', size=20)

In [17]:
df_cap_thickness_group = Combined_df.groupby(['ThicknessAlGaNBarrier']) # Group by thickness

fig_ncols, fig_nrows = 3, 3
for name, group in df_cap_thickness_group:
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    fig, axs = plt.subplots(fig_ncols, fig_nrows, figsize=(16.5,14), 
                            sharey=True,constrained_layout=True)
    show_colorbar = False
    tick_multiplicator=[0.1, 0.05, 0.1, 0.05]
    for ii in range(fig_ncols*fig_nrows):
        if ii < len(plot_mobilities):
            which_mobility = plot_mobilities[ii]
        else:
            axs[ii//3][ii%3].axis("off") 
            continue
        x_label_text, y_label_text, z_label = '','',''
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]

        vmin = Combined_df[which_mobility].min()+1e-5 if which_mobility in ['2DEG', 'LFOM', 'LFOMnorm'] else 1
        vmax =  Combined_df[which_mobility].max() 
        #vmin, vmax = zz.min()+1e-20, zz.max()
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['LFOMnorm_'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
        #print(ii, ii%3)
        if ii-6>=0: x_label_text = 'Al composition channel'
        if ii%3==0: y_label_text = 'Al composition barrier'
        if ii%3-2==0: z_label=r'$\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
        if which_mobility == 'LFOM': z_label=r'LFOM (MW/cm$^2$)'
        if which_mobility == 'LFOMnorm': z_label=r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{GaN}$'
        if which_mobility == '2DEG': z_label=r'2DEG ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)'

        ##### Plot composition map 
        fig, _ = lpltq3d.Plotq3D(xx,yy,zz, fig=fig, ax=axs[ii//3][ii%3], 
                                                vmin=vmin, vmax=vmax, 
                                                x_label=x_label_text, y_label=y_label_text,
                                                z_label=z_label, interpolation_method='linear',
                                                interpolation_points = 20,
                                                tick_multiplicator=tick_multiplicator,
                                                title_label=which_mobility[:], 
                                                cbar_mappable=cbar_mapable, norm=norm,
                                                show_contour_lines=False, 
                                                marker='s', marker_size=24**2,
                                                cbar_text=z_label,show_colorbar=True,
                                                plot_controur=False, plot_scatter=True, 
                                                savefigure=False, show_plot=False)
        plt.suptitle(f"Barrier thickness = {barrier_thickness} nm", size=24)
        if ii%3 != 0: axs[ii//3][ii%3].get_yaxis().set_visible(False)
    lpltq3d.save_figure(f'Barrier_{barrier_thickness}.png', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=75)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting IFR...
	-- Plotting 